# <center> ETL (Parler) </center>
<center> Andrea Šipka, Social Computing Group, University of Zurich </center>

## Imports and settings

In [367]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', 300)
pd.options.display.max_rows = 999
pd.options.mode.chained_assignment = None
plt.rcParams.update({'font.size': 16})
plt.style.use('bmh')

### Only keep lines that have data, removing system messages

In [28]:
with open('data/stopthesteal.txt') as oldfile, open('data/stopthesteal_clean.json', 'w') as newfile:
    for line in oldfile:
        if line[:8] == '{"_id":"':       
            if '[network]' in line:
                newfile.write(line[:line.find('[network]')]+'\n')
            else:
                newfile.write(line[:-2]+'\n')

In [88]:
with open('data/stopthesteal_latest.txt') as oldfile, open('data/stopthesteal_latest_clean.json', 'w') as newfile:
    for line in oldfile:
        if line[:8] == '{"_id":"':       
            if '[network]' in line:
                newfile.write(line[:line.find('[network]')]+'\n')
            else:
                newfile.write(line[:-2]+'\n')

### NB: You may need to manually remove a final newline and add a closing curly bracket in the json file created above

TODO: Automate

In [119]:
data = pd.read_json('data/stopthesteal_clean.json', lines=True)

In [120]:
data_latest = pd.read_json('data/stopthesteal_latest_clean.json', lines=True)

### Get the data about authors from json within json

In [123]:
userdata = pd.json_normalize(data.creator)
userdata = userdata.add_prefix('creator_')
userdata.shape

(286354, 21)

In [124]:
userdata_latest = pd.json_normalize(data_latest.creator)
userdata_latest = userdata_latest.add_prefix('creator_')
userdata_latest.shape

(92179, 21)

In [125]:
parler_data = pd.concat([data,userdata], axis=1)

In [126]:
parler_data_latest = pd.concat([data_latest,userdata_latest], axis=1)

### Merge recent data with old data

In [130]:
data = pd.concat([parler_data_latest[parler_data_latest.createdAt > 20201126142233], parler_data])

In [131]:
data.shape

(321393, 42)

### Save to CSV

In [132]:
data.to_csv('data/stopthesteal_parsed.csv')

## Getting features cleaned up

In [178]:
data = pd.read_csv('data/stopthesteal_parsed.csv')

/Users/andreasipka/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [179]:
data.head()

,Unnamed: 0,_id,@,article,body,comments,createdAt,creator,depth,depthRaw,hashtags,id,impressions,links,preview,reposts,shareLink,sensitive,state,upvotes,parent,root,creator_id,creator_bio,creator_blocked,creator_coverPhoto,creator_followed,creator_human,creator_integration,creator_joined,creator_muted,creator_name,creator_rss,creator_private,creator_profilePhoto,creator_username,creator_verified,creator_verifiedComments,creator_badges,creator_score,creator_interactions,creator_state,creator_subscribed
0,0,461b4cc3118142d2958a58ddeaa9f320,{},0.0,So #sleepyjoe twist his ankle while playing wi...,0,20201129224519,"{'id': 'd8f57880bbd540e9a1e1bc89e46fdcef', 'bi...",0,0,"['sleepyjoe', 'trumpwon2020', 'stopthesteal', ...",461b4cc3118142d2958a58ddeaa9f320,0,[],So #sleepyjoe twist his ankle while playing wi...,0,https://parler.com/post/461b4cc3118142d2958a58...,False,4.0,0,NaN,NaN,d8f57880bbd540e9a1e1bc89e46fdcef,"I am a patriot, and proud supporter of the bes...",False,f74d523f9d8e403f81f4587da9c24c4f,False,False,False,20201001121704,False,Haley S.,False,False,82c3df8812b149ae87354125efdc6ccf,Absolutefusteration,False,False,[],117,258,1.0,NaN
1,1,b5c7b951ea1d485db3b490b8f07f56be,{},0.0,"""God Bless America"" Flash Mob with Denver Bras...",0,20201129224516,"{'id': '5ef511800e6e48d8b23e92d37467b718', 'bi...",0,0,"['jesusislord', 'trump2020', 'jesusmatters', '...",b5c7b951ea1d485db3b490b8f07f56be,0,"[{'_id': 'luzWb', 'createdAt': '20190717100132...","""God Bless America"" Flash Mob with Denver Bras...",0,https://parler.com/post/b5c7b951ea1d485db3b490...,False,4.0,0,NaN,NaN,5ef511800e6e48d8b23e92d37467b718,Faith✝️Family👨‍👩‍👧‍👦America🦅Trump🇺🇸Freedom💪Pro...,False,12d5f5bf38834ea7b6613ba4081096eb,False,False,False,20200918173929,False,Cascadefalls,False,False,0f077b2a7fa044e8a5f9e8856d04bdd6,Cascadefalls,False,False,[],3.0k,258,1.0,NaN
2,2,fe93a766737745188f5fae4a5339b186,{},0.0,"#donaldtrump, #trump, #presidenttrump, #teamtr...",0,20201129224513,"{'id': '765cdf14184243259ad5fddb71a854ce', 'bi...",0,0,"['donaldtrump', 'trump', 'presidenttrump', 'te...",fe93a766737745188f5fae4a5339b186,0,"[{'_id': 'E3NUG', 'createdAt': '20201129224513...","#donaldtrump, #trump, #presidenttrump, #teamtr...",0,https://parler.com/post/fe93a766737745188f5fae...,False,4.0,0,NaN,NaN,765cdf14184243259ad5fddb71a854ce,The vote is our primary right upon which all o...,False,25dc373993c64e75a8001fbc2d316468,False,True,False,20201107210729,False,Jack Jones,False,False,fda1bc735d4844baa88ada3ea62d549d,1965jackjones,False,False,[0],1.5k,3,1.0,NaN
3,3,124139704dc64dac9a0ba1c57587e348,{},0.0,#electionfraud #fraud #stopthesteal,0,20201129224510,"{'id': 'a32f450ba92d4a9486b0fb1b94ee397f', 'bi...",1,1,"['electionfraud', 'fraud', 'stopthesteal']",124139704dc64dac9a0ba1c57587e348,0,[],#electionfraud #fraud #stopthesteal,0,https://parler.com/post/124139704dc64dac9a0ba1...,False,4.0,0,"{'_id': '47e76823f7304f84a1898d70ac460ba1', '@...",NaN,a32f450ba92d4a9486b0fb1b94ee397f,All races were created equal by God!\n#trump2020,False,6c0f247a2d0b4fbea0ec51546fe63c94,False,True,False,20201108222023,False,President Elect Poppakap,False,False,2a8c272773f249678cfa2cf758908944,Poppakap,False,False,[0],2.5k,259,1.0,NaN
4,4,9733b4c526d845679b211d4fc72234ea,{},0.0,Judge Awards Sasha and Malia Obama’s Biologica...,0,20201129224453,"{'id': '2a5c8defed0648c49c0742a7582ac797', 'bi...",0,0,"['blm', 'dnc', 'trumptrain2020', 'freedom', 'f...",9733b4c526d845679b211d4fc72234ea,0,"[{'_id': 'DEnvb', 'createdAt': '20201125214939...",Judge Awards Sasha and Malia Obama’s Biologica...,0,https://parler.com/post/9733b4c526d845679b211d...,False,4.0,0,NaN,NaN,2a5c8defed0648c49c0742a7582ac797,Judge Jeanine Pirro Fan Page,False,3d5b9bc72ee0463ba14a7b34c8a1e7db,False,False,False,20201124184003,False,Judge Jeanine Pirro,False,False,d4968d0b26f643daae7d3bd3975687c6,judgejeaninepirrofanpage,False,False,[],1.8k,258,1.0,NaN


## Extract tags

In [91]:
def make_mentions_df(df):
    
    list_to_save = []
    
    for index, row in df.iterrows():
        
        # only do those who have some tags
        if row['@'] != '{}':
            
            # split the string into separate mentions and extract each
            for item in row['@'][1:-1].split(', '):
                list_to_save.append([row['_id'], 
                                     row['creator_id'], 
                                     row['creator_username'], 
                                     item[item.find(':') + 3 : -1], # this extracts tagged user id
                                     item[1 : item.find('\':')]]) ## this extracts tagged username
            
    return pd.DataFrame(list_to_save, columns=['post_id', 'creator_id', 'creator_username', 'tagged_user_id', 'tagged_username'])

In [96]:
mentions_data = make_mentions_df(data)

In [97]:
mentions_data.head()

,post_id,creator_id,creator_username,tagged_user_id,tagged_username
0,7dc7522299a046e0b67bc45e0112efe0,11099c81c1ca42f88a977b8d4e5e2336,AlexJonesWins,f1c6851c95c98c2d2af35eec0c9cc635,ali
1,7dc7522299a046e0b67bc45e0112efe0,11099c81c1ca42f88a977b8d4e5e2336,AlexJonesWins,dcdd1dc516c64d7880a54edb37bdb8ef,alexjones
2,07ac00ecbc7f4c47bda5ca417c03ed7b,aedaffa4fff44d7bb44d3784817a6af5,Sparry131,89e43877e52645a1bf8239d8a4fa2ac2,genflynn
3,07ac00ecbc7f4c47bda5ca417c03ed7b,aedaffa4fff44d7bb44d3784817a6af5,Sparry131,fd4eaf2436164605b291ba98e7061d7b,sidneypowell
4,07ac00ecbc7f4c47bda5ca417c03ed7b,aedaffa4fff44d7bb44d3784817a6af5,Sparry131,2ea66900ed69a5005fe51fc8b07a1711,linwood


In [98]:
mentions_data.shape

(110716, 5)

In [99]:
mentions_data.to_csv('data/mentions_data.csv', index =  False)

### Clean up hashtag list, figure out how many hashtags were used

In [181]:
def parse_hashtags(s):
    
    hashtag_list = s['hashtags'][2:-2].split('\', \'')
        
    s['nr_hashtags'] = len(hashtag_list)
    s['hashtags_used'] = ', '.join(hashtag_list)
    
    return s

In [182]:
data['hashtags_used'] = ''
data['nr_hashtags'] = 0

data = data.apply(parse_hashtags, axis=1)

### Get rid of pointless columns and rename columns so that they make more sense

In [185]:
# delete unnecessary columns
del data['Unnamed: 0'] # consequence of exporting importing
del data['_id'] # same as id
del data['creator'] # already extracted
del data['depthRaw'] # same as depth
del data['@'] # already extracted
del data['hashtags'] # already extracted
del data['preview'] # subset of whats in body
del data['state'] # always 4 for all the rows
del data['creator_blocked'] # individual to account used to scrape
del data['creator_followed'] # individual to account used to scrape
del data['creator_muted'] # individual to account used to scrape
del data['creator_subscribed'] # individual to account used to scrape
del data['creator_private'] # not as we could see

In [187]:
# rename columns
data.columns = ['article', 'body', 'comments', 'post_date', 'depth', 'post_id',
       'impressions', 'links', 'reposts', 'shareLink', 'sensitive', 'upvotes',
       'parent', 'root', 'creator_id', 'creator_bio', 'creator_coverPhoto',
       'creator_human', 'creator_integration', 'creator_joined',
       'creator_name', 'creator_rss', 'creator_profilePhoto',
       'creator_username', 'creator_verified', 'creator_verifiedComments',
       'creator_badges', 'creator_score', 'creator_interactions',
       'creator_state', 'hashtags', 'nr_hashtags']

### Calculate the number of mentions (tagging other users) each post has

In [212]:
mentions_per_post = mentions_data.groupby(['post_id']).size().reset_index(name="nr_mentions")
data = data.merge(mentions_per_post, left_on='post_id', right_on='post_id', how='left')
data.nr_mentions.fillna(0, inplace=True)

### Fix data types

In [236]:
data.article = data.article.astype('Int64')
data.depth = data.depth.astype('Int64')
data.creator_interactions = data.creator_interactions.astype('Int64')
data.creator_state = data.creator_state.astype('Int64')
data.nr_mentions = data.nr_mentions.astype('Int64')

In [239]:
data['creator_joined'] =  pd.to_datetime(data['creator_joined'], format='%Y%m%d%H%M%S')
data['createdAt'] =  pd.to_datetime(data['createdAt'], format='%Y%m%d%H%M%S')

### deal with parler using text to describe numbers - e.g. convert 2.6k to 2600

In [388]:
def str_to_num(str_in):
    try:
        # simple intiger
        string_int = int(str_in)
        return int(str_in)
    except ValueError:
        # thousands
        if 'k' in str_in:
            return math.ceil(float(str_in[:-1])*1000)
        # millions
        elif 'm' in str_in:
            return math.ceil(float(str_in[:-1])*1000000)
        else:
            return np.NaN
    
def cleanup_numbers(s):
    s['comments_clean'] = str_to_num(s['comments'])
    s['impressions_clean'] = str_to_num(s['impressions'])
    s['reposts_clean'] = str_to_num(s['reposts'])
    s['upvotes_clean'] = str_to_num(s['upvotes'])
    s['creator_score_clean'] = str_to_num(s['creator_score'])
    return s

In [369]:
# Run the function to convert tricky strings to numbers
data = data.apply(cleanup_numbers, axis=1)

In [370]:
# convert new columns to ints
data.comments_clean = data.comments_clean.astype('Int64')
data.reposts_clean = data.reposts_clean.astype('Int64')
data.upvotes_clean = data.upvotes_clean.astype('Int64')
data.creator_score_clean = data.creator_score_clean.astype('Int64')
data.impressions_clean = data.impressions_clean.astype('Int64')

In [385]:
# delete unnecessary columns
del data['comments']
del data['reposts']
del data['upvotes']
del data['creator_score']
del data['impressions']

In [387]:
# rename columns
data.columns = ['article', 'body', 'post_date', 'depth', 'post_id', 'links',
       'shareLink', 'sensitive', 'parent', 'root', 'creator_id', 'creator_bio',
       'creator_coverPhoto', 'creator_human', 'creator_integration',
       'creator_joined', 'creator_name', 'creator_rss', 'creator_profilePhoto',
       'creator_username', 'creator_verified', 'creator_verifiedComments',
       'creator_badges', 'creator_interactions', 'creator_state', 'hashtags',
       'nr_hashtags', 'nr_mentions', 'comments', 'impressions',
       'reposts', 'upvotes', 'creator_score']

### reorder columns so that it makes more sense

In [401]:
data = data[['post_id', 'body', 'post_date', 'article', 'depth', 'links',
       'shareLink', 'sensitive', 'parent', 'root', 'creator_id', 'creator_bio',
       'creator_coverPhoto', 'creator_human', 'creator_integration',
       'creator_joined', 'creator_name', 'creator_rss', 'creator_profilePhoto',
       'creator_username', 'creator_verified', 'creator_verifiedComments',
       'creator_badges', 'creator_interactions', 'creator_state', 'creator_score', 
       'hashtags', 'nr_hashtags', 'nr_mentions', 'comments', 'impressions',
       'reposts', 'upvotes']]

### create features for badges

In [425]:
data['creator_badge_0'] = np.where(data['creator_badges'].str.contains('0'), True, False)
data['creator_badge_1'] = np.where(data['creator_badges'].str.contains('1'), True, False)
data['creator_badge_2'] = np.where(data['creator_badges'].str.contains('2'), True, False)
data['creator_badge_3'] = np.where(data['creator_badges'].str.contains('3'), True, False)
data['creator_badge_4'] = np.where(data['creator_badges'].str.contains('4'), True, False)
data['creator_badge_5'] = np.where(data['creator_badges'].str.contains('5'), True, False)
data['creator_badge_6'] = np.where(data['creator_badges'].str.contains('6'), True, False)
data['creator_badge_7'] = np.where(data['creator_badges'].str.contains('7'), True, False)
data['creator_badge_9'] = np.where(data['creator_badges'].str.contains('9'), True, False)

In [431]:
del data['creator_badges']

In [433]:
data.shape

(321393, 41)

### Extract links

In [542]:
def make_link_df(s):
    
    list_to_save = []
    
    for index, row in s.iterrows():
        
        list_of_lists_links = row['links'][1:-1].split('}, {')

        for link_list in list_of_lists_links:
            
            s_link_id = s_createdAt = s_domain = s_full_link = s_type = s_locale = s_site_name = s_state = ''
            
            for item in ''.join( c for c in link_list if  c not in '\{\}').split(', '):

                if '\'_id\'' in item[:item.find(':')]:
                    s_link_id = item[item.find('\'_id\': \'')+len('\'_id\': \'') : -1]

                if '\'createdAt\'' in item[:item.find(':')]:
                    s_createdAt = item[item.find('\'createdAt\': \'')+len('\'createdAt\': \'') : -1]

                if '\'domain\'' in item[:item.find(':')]:
                    s_domain = item[item.find('\'domain\': \'')+len('\'domain\': \'') : -1]

                if '\'long\'' in item[:item.find(':')]:
                    s_full_link = item[item.find('\'long\': \'')+len('\'long\': \'') : -1]

                if '\'type\'' in item[:item.find(':')]:
                    s_type = item[item.find('\'type\': \'')+len('\'type\': \'') : -1]

                if '\'locale\'' in item[:item.find(':')]:
                    s_locale = item[item.find('\'locale\': \'')+len('\'locale\': \'') : -1]

                if '\'site_name\'' in item[:item.find(':')]:
                    s_site_name = item[item.find('\'site_name\': \'')+len('\'site_name\': \'') : -1]

                if '\'state\'' in item[:item.find(':')]:
                    s_state = item[item.find('\'state\': \'')+len('\'state\': \'') : -1]
                
            list_to_save.append([row['post_id'], row['post_date'], row['creator_id'], 
                                 s_link_id, s_createdAt, s_domain, s_full_link, s_type, 
                                 s_locale, s_site_name, s_state])
            

    return pd.DataFrame(list_to_save, columns=['post_id', 'post_date', 'creator_id', 
                                               'link_id', 'createdAt', 'domain', 'full_link', 
                                               'type','locale', 'site_name', 'state'])

In [544]:
# only get stuff with links
link_data = make_link_df(data[data.links != '[]'])

In [547]:
link_data['createdAt'] =  pd.to_datetime(link_data['createdAt'], format='%Y%m%d%H%M%S')

In [549]:
link_data.shape

(199011, 11)

In [550]:
link_data.to_csv('data/link_data.csv', index=False)

### add number of links per video to the main datatable

In [551]:
links_per_post = link_data.groupby(['post_id']).size().reset_index(name="nr_links")

In [554]:
data = data.merge(links_per_post, left_on='post_id', right_on='post_id', how='left')
data.nr_links.fillna(0, inplace=True)

In [556]:
data.nr_links = data.nr_links.astype('Int64')

In [557]:
data.head()

,post_id,body,post_date,article,depth,links,shareLink,sensitive,parent,root,creator_id,creator_bio,creator_coverPhoto,creator_human,creator_integration,creator_joined,creator_name,creator_rss,creator_profilePhoto,creator_username,creator_verified,creator_verifiedComments,creator_interactions,creator_state,creator_score,hashtags,nr_hashtags,nr_mentions,comments,impressions,reposts,upvotes,creator_badge_0,creator_badge_1,creator_badge_2,creator_badge_3,creator_badge_4,creator_badge_5,creator_badge_6,creator_badge_7,creator_badge_9,nr_links
0,461b4cc3118142d2958a58ddeaa9f320,So #sleepyjoe twist his ankle while playing wi...,2020-11-29 22:45:19,0,0,[],https://parler.com/post/461b4cc3118142d2958a58...,False,NaN,NaN,d8f57880bbd540e9a1e1bc89e46fdcef,"I am a patriot, and proud supporter of the bes...",f74d523f9d8e403f81f4587da9c24c4f,False,False,2020-10-01 12:17:04,Haley S.,False,82c3df8812b149ae87354125efdc6ccf,Absolutefusteration,False,False,258,1,117,"sleepyjoe, trumpwon2020, stopthesteal, magaagain",4,0,0,0,0,0,False,False,False,False,False,False,False,False,False,0
1,b5c7b951ea1d485db3b490b8f07f56be,"""God Bless America"" Flash Mob with Denver Bras...",2020-11-29 22:45:16,0,0,"[{'_id': 'luzWb', 'createdAt': '20190717100132...",https://parler.com/post/b5c7b951ea1d485db3b490...,False,NaN,NaN,5ef511800e6e48d8b23e92d37467b718,Faith✝️Family👨‍👩‍👧‍👦America🦅Trump🇺🇸Freedom💪Pro...,12d5f5bf38834ea7b6613ba4081096eb,False,False,2020-09-18 17:39:29,Cascadefalls,False,0f077b2a7fa044e8a5f9e8856d04bdd6,Cascadefalls,False,False,258,1,3000,"jesusislord, trump2020, jesusmatters, bluelive...",58,0,0,0,0,0,False,False,False,False,False,False,False,False,False,1
2,fe93a766737745188f5fae4a5339b186,"#donaldtrump, #trump, #presidenttrump, #teamtr...",2020-11-29 22:45:13,0,0,"[{'_id': 'E3NUG', 'createdAt': '20201129224513...",https://parler.com/post/fe93a766737745188f5fae...,False,NaN,NaN,765cdf14184243259ad5fddb71a854ce,The vote is our primary right upon which all o...,25dc373993c64e75a8001fbc2d316468,True,False,2020-11-07 21:07:29,Jack Jones,False,fda1bc735d4844baa88ada3ea62d549d,1965jackjones,False,False,3,1,1500,"donaldtrump, trump, presidenttrump, teamtrump,...",16,0,0,0,0,0,True,False,False,False,False,False,False,False,False,1
3,124139704dc64dac9a0ba1c57587e348,#electionfraud #fraud #stopthesteal,2020-11-29 22:45:10,0,1,[],https://parler.com/post/124139704dc64dac9a0ba1...,False,"{'_id': '47e76823f7304f84a1898d70ac460ba1', '@...",NaN,a32f450ba92d4a9486b0fb1b94ee397f,All races were created equal by God!\n#trump2020,6c0f247a2d0b4fbea0ec51546fe63c94,True,False,2020-11-08 22:20:23,President Elect Poppakap,False,2a8c272773f249678cfa2cf758908944,Poppakap,False,False,259,1,2500,"electionfraud, fraud, stopthesteal",3,0,0,0,0,0,True,False,False,False,False,False,False,False,False,0
4,9733b4c526d845679b211d4fc72234ea,Judge Awards Sasha and Malia Obama’s Biologica...,2020-11-29 22:44:53,0,0,"[{'_id': 'DEnvb', 'createdAt': '20201125214939...",https://parler.com/post/9733b4c526d845679b211d...,False,NaN,NaN,2a5c8defed0648c49c0742a7582ac797,Judge Jeanine Pirro Fan Page,3d5b9bc72ee0463ba14a7b34c8a1e7db,False,False,2020-11-24 18:40:03,Judge Jeanine Pirro,False,d4968d0b26f643daae7d3bd3975687c6,judgejeaninepirrofanpage,False,False,258,1,1800,"blm, dnc, trumptrain2020, freedom, facebook, l...",64,0,0,0,0,0,False,False,False,False,False,False,False,False,False,1


### save cleaned data

In [558]:
data.to_csv('data/stopthesteal_parsed.csv', index=False)

## Extract hashtags

In [447]:
def make_hashtag_dataframe(df):
        
    list_to_save = []
    
    for index, row in df.iterrows():
        for hashtag in row['hashtags'].split(', '):
            list_to_save.append([row['post_id'], row['creator_id'], row['post_date'], hashtag])
            
    return pd.DataFrame(list_to_save, columns=['post_id', 'creator_id', 'post_date', 'hashtag'])

In [448]:
hashtag_data = make_hashtag_dataframe(data)

In [449]:
hashtag_data.head()

,post_id,creator_id,post_date,hashtag
0,461b4cc3118142d2958a58ddeaa9f320,d8f57880bbd540e9a1e1bc89e46fdcef,2020-11-29 22:45:19,sleepyjoe
1,461b4cc3118142d2958a58ddeaa9f320,d8f57880bbd540e9a1e1bc89e46fdcef,2020-11-29 22:45:19,trumpwon2020
2,461b4cc3118142d2958a58ddeaa9f320,d8f57880bbd540e9a1e1bc89e46fdcef,2020-11-29 22:45:19,stopthesteal
3,461b4cc3118142d2958a58ddeaa9f320,d8f57880bbd540e9a1e1bc89e46fdcef,2020-11-29 22:45:19,magaagain
4,b5c7b951ea1d485db3b490b8f07f56be,5ef511800e6e48d8b23e92d37467b718,2020-11-29 22:45:16,jesusislord


In [450]:
hashtag_data.shape

(4282346, 4)

In [451]:
hashtag_data.to_csv('data/hashtag_data.csv', index =  False)

## Extract user information

Note on this dropping duplicates: as this dataset has a temporal dimension, some users will have a different number of followers, badges etc overtime. But drop duplicates will save the first row it finds, which will be the latest post, and therefore the most up to date one. Of course, for more precise current information, we would need to query parler for user stats.

In [562]:
users = data.drop_duplicates(subset=['creator_id'])

In [563]:
users.shape

(44909, 42)

In [564]:
users.index = pd.to_datetime(users.creator_joined)

In [565]:
# only keep columns I need because I am mildly OCD
users = users[['creator_id', 'creator_bio',
       'creator_coverPhoto', 'creator_human', 'creator_integration',
       'creator_joined', 'creator_name', 'creator_rss', 'creator_profilePhoto',
       'creator_username', 'creator_verified', 'creator_verifiedComments',
       'creator_interactions', 'creator_state', 'creator_score', 'creator_badge_0', 
       'creator_badge_1', 'creator_badge_2',
       'creator_badge_3', 'creator_badge_4', 'creator_badge_5',
       'creator_badge_6', 'creator_badge_7', 'creator_badge_9']]

In [566]:
# Get mean values for numeric indicators of post success
user_posts_table = data.groupby('creator_id')[['nr_hashtags', 'nr_mentions', 'nr_links', 
                  'comments', 'impressions', 'reposts', 'upvotes']].agg('mean').reset_index()

In [567]:
# Get number of posts in dataset per user
user_post_no = data['creator_id'].value_counts().to_frame()
user_post_no.columns = ['nr_posts']
user_posts_table = user_posts_table.merge(user_post_no, left_on='creator_id', right_index = True)

In [568]:
# get number of posts that are echos
data['echo_numeric'] = np.where(data['depth'] == 0, 0, 1)
no_echos = data.groupby('creator_id')[['echo_numeric']].agg('sum').reset_index()
user_posts_table = user_posts_table.merge(no_echos, left_on='creator_id', right_on = 'creator_id')

In [569]:
# rename columns
user_posts_table.columns = ['creator_id', 'nr_hashtags', 'nr_mentions', 'nr_links', 'mean_comments',
       'mean_impressions', 'mean_reposts', 'mean_upvotes', 'nr_posts', 'nr_echos']

In [570]:
# calculate echo to post ration
user_posts_table['echo_ratio'] = user_posts_table['nr_echos'] / user_posts_table['nr_posts']

In [571]:
users = users.merge(user_posts_table, left_on='creator_id', right_on = 'creator_id')

In [572]:
users.shape

(44909, 34)

In [573]:
users.head()

,creator_id,creator_bio,creator_coverPhoto,creator_human,creator_integration,creator_joined,creator_name,creator_rss,creator_profilePhoto,creator_username,creator_verified,creator_verifiedComments,creator_interactions,creator_state,creator_score,creator_badge_0,creator_badge_1,creator_badge_2,creator_badge_3,creator_badge_4,creator_badge_5,creator_badge_6,creator_badge_7,creator_badge_9,nr_hashtags,nr_mentions,nr_links,mean_comments,mean_impressions,mean_reposts,mean_upvotes,nr_posts,nr_echos,echo_ratio
0,d8f57880bbd540e9a1e1bc89e46fdcef,"I am a patriot, and proud supporter of the bes...",f74d523f9d8e403f81f4587da9c24c4f,False,False,2020-10-01 12:17:04,Haley S.,False,82c3df8812b149ae87354125efdc6ccf,Absolutefusteration,False,False,258,1.0,117,False,False,False,False,False,False,False,False,False,3.185185,0.000000,0.037037,0.074074,108.370370,0.000000,0.703704,27,2,0.074074
1,5ef511800e6e48d8b23e92d37467b718,Faith✝️Family👨‍👩‍👧‍👦America🦅Trump🇺🇸Freedom💪Pro...,12d5f5bf38834ea7b6613ba4081096eb,False,False,2020-09-18 17:39:29,Cascadefalls,False,0f077b2a7fa044e8a5f9e8856d04bdd6,Cascadefalls,False,False,258,1.0,3000,False,False,False,False,False,False,False,False,False,57.216667,0.001515,0.298485,0.224242,687.254545,0.922727,2.227273,660,464,0.703030
2,765cdf14184243259ad5fddb71a854ce,The vote is our primary right upon which all o...,25dc373993c64e75a8001fbc2d316468,True,False,2020-11-07 21:07:29,Jack Jones,False,fda1bc735d4844baa88ada3ea62d549d,1965jackjones,False,False,3,1.0,1500,True,False,False,False,False,False,False,False,False,11.448000,0.032000,0.988000,0.140000,316.320000,0.688000,1.592000,250,5,0.020000
3,a32f450ba92d4a9486b0fb1b94ee397f,All races were created equal by God!\n#trump2020,6c0f247a2d0b4fbea0ec51546fe63c94,True,False,2020-11-08 22:20:23,President Elect Poppakap,False,2a8c272773f249678cfa2cf758908944,Poppakap,False,False,259,1.0,2500,True,False,False,False,False,False,False,False,False,6.600000,0.050000,0.450000,0.525000,444.900000,1.225000,2.350000,40,20,0.500000
4,2a5c8defed0648c49c0742a7582ac797,Judge Jeanine Pirro Fan Page,3d5b9bc72ee0463ba14a7b34c8a1e7db,False,False,2020-11-24 18:40:03,Judge Jeanine Pirro,False,d4968d0b26f643daae7d3bd3975687c6,judgejeaninepirrofanpage,False,False,258,1.0,1800,False,False,False,False,False,False,False,False,False,65.319149,0.007092,0.794326,1.617021,2006.255319,6.035461,9.177305,141,29,0.205674


In [574]:
users.to_csv('data/users_data.csv', index =  False)

## Export hashtag coocurence data for network analysis

In [579]:
from itertools import combinations

def make_cooc_df(df):
        
    list_to_save = []
    
    for index, row in df.iterrows():
        hashtaglist = row['hashtags'].split(', ')
        
        for h1, h2 in combinations(hashtaglist, 2):
            if h1 < h2:
                list_to_save.append([h1, h2, row['creator_id']])
            else:
                list_to_save.append([h2, h1, row['creator_id']])
            
    return pd.DataFrame(list_to_save, columns=['hashtag1', 'hashtag2', 'creator_id'])

In [580]:
hashtag_cooc_data = make_cooc_df(data)

In [581]:
hashtag_cooc_data.head()

,hashtag1,hashtag2,creator_id
0,sleepyjoe,trumpwon2020,d8f57880bbd540e9a1e1bc89e46fdcef
1,sleepyjoe,stopthesteal,d8f57880bbd540e9a1e1bc89e46fdcef
2,magaagain,sleepyjoe,d8f57880bbd540e9a1e1bc89e46fdcef
3,stopthesteal,trumpwon2020,d8f57880bbd540e9a1e1bc89e46fdcef
4,magaagain,trumpwon2020,d8f57880bbd540e9a1e1bc89e46fdcef


In [582]:
hashtag_cooc_data.shape

(80735677, 3)

In [583]:
hashtag_cooc_weighted = hashtag_cooc_data.groupby(["hashtag1", "hashtag2"]).size().reset_index(name="co_occurences")

In [587]:
hashtag_cooc_weighted.sample(10)

,hashtag1,hashtag2,co_occurences
1744884,quanon,trumpisourpresident,1
1581340,neverdips,youseek,1
1898574,thereformparty,trump,1
1825978,skynewsaustralia,stopcommunism,1
1296895,ingodwetrust,treason,5
1900841,thetruthwillprevail,trump2020landslide,30
31330,223,releasethekraken,2
890999,electionfraud,oneworldorder,4
1472348,maga,rose,4
446158,catholicdetroit,joebiden,1


In [588]:
hashtag_cooc_weighted.shape

(1965290, 3)

In [596]:
hashtag_cooc_weighted.to_csv('data/hashtag_cooc_full.csv', index =  False)

In [595]:
hashtag_cooc_weighted[hashtag_cooc_weighted.co_occurences > 5].shape

(433525, 3)

In [597]:
hashtag_cooc_weighted[hashtag_cooc_weighted.co_occurences > 5].to_csv('data/hashtag_cooc_small.csv', index =  False)

### my 100 favourite pairs

In [601]:
hashtag_cooc_weighted[hashtag_cooc_weighted.co_occurences > 99].shape

(111495, 3)

## ToDo list
* extract parent field
* extract root field